In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

22/12/06 15:21:54 WARN Utils: Your hostname, diakites-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.43.232 instead (on interface en0)
22/12/06 15:21:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/06 15:21:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

In [4]:
path = "/Users/diakite/Downloads/random_objectif_hebdomadaire_grossiste_202211.csv"

In [5]:
df = pd.read_csv(path, sep =";")
df.head()

,COMMISSION_AGENT_REEL,COMMISSION_GROSSISTE_FIXE,COMMISSION_GROSSISTE_REEL,COMPTE_PDV,COMPTE_REVENDEUR,LAST_DAY_OF_WEEK,OBJECTIF_AGENT,OBJECTIF_GROSSISTE,OBJECTIF_REALISE_AGENT,OBJECTIF_REALISE_GROSSISTE,OBJECTIF_SEMAINE,PRIME_HEBDOMADAIRE_GROSSISTE,R/O_AGENT,R/O_GROSSISTE,RUNTIME,SEMAINE,TETE_DE_PONT,TRANSACTION_TAG
0,NaN,6050129.0,151253,NaN,NaN,20221107,NaN,NaN,NaN,1638919763,1559197306,151253,NaN,1,08/11/2022 11:20:11,1,788280641,CASHOUT
1,NaN,6698091.0,167452,NaN,NaN,20221107,NaN,NaN,NaN,4302822888,4279525010,167452,NaN,1,08/11/2022 11:20:11,1,708419204,CASHIN
2,NaN,309630.0,7741,NaN,NaN,20221107,NaN,NaN,NaN,34428550,33999050,7741,NaN,1,08/11/2022 11:20:11,1,707068208,CASHOUT
3,NaN,396651.0,9916,NaN,NaN,20221107,NaN,NaN,NaN,23939600,23680296,9916,NaN,1,08/11/2022 11:20:11,1,702155864,CASHOUT
4,NaN,562175.0,14054,NaN,NaN,20221107,NaN,NaN,NaN,85874875,68283889,14054,NaN,1,08/11/2022 11:20:11,1,747118290,CASHIN


In [6]:
df_spark = spark.read.option("delimiter", ";").option("header","true").csv(path)
df_spark.show(5)

+---------------------+-------------------------+-------------------------+----------+----------------+----------------+--------------+------------------+----------------------+--------------------------+----------------+----------------------------+---------+-------------+-------------------+-------+------------+---------------+
|COMMISSION_AGENT_REEL|COMMISSION_GROSSISTE_FIXE|COMMISSION_GROSSISTE_REEL|COMPTE_PDV|COMPTE_REVENDEUR|LAST_DAY_OF_WEEK|OBJECTIF_AGENT|OBJECTIF_GROSSISTE|OBJECTIF_REALISE_AGENT|OBJECTIF_REALISE_GROSSISTE|OBJECTIF_SEMAINE|PRIME_HEBDOMADAIRE_GROSSISTE|R/O_AGENT|R/O_GROSSISTE|            RUNTIME|SEMAINE|TETE_DE_PONT|TRANSACTION_TAG|
+---------------------+-------------------------+-------------------------+----------+----------------+----------------+--------------+------------------+----------------------+--------------------------+----------------+----------------------------+---------+-------------+-------------------+-------+------------+---------------+
|   

In [7]:
df_spark = df_spark.select("COMMISSION_GROSSISTE_FIXE", "COMMISSION_AGENT_REEL", "TRANSACTION_TAG", "TETE_DE_PONT")
df_spark.show()


+-------------------------+---------------------+---------------+------------+
|COMMISSION_GROSSISTE_FIXE|COMMISSION_AGENT_REEL|TRANSACTION_TAG|TETE_DE_PONT|
+-------------------------+---------------------+---------------+------------+
|                6050129.0|                 null|        CASHOUT|  0788280641|
|                6698091.0|                 null|         CASHIN|  0708419204|
|                 309630.0|                 null|        CASHOUT|  0707068208|
|                 396651.0|                 null|        CASHOUT|  0702155864|
|                 562175.0|                 null|         CASHIN|  0747118290|
|              1.0301494E7|                 null|         CASHIN|  0758023975|
|                 268640.0|                 null|         CASHIN|  0707068367|
|                6698091.0|                 null|        CASHOUT|  0708419204|
|                  24002.0|                 null|        CASHOUT|  0747118393|
|                 549267.0|                 null|   

In [19]:
df_spark.groupBy("TRANSACTION_TAG").count().explain(True)  


#.show() \


== Parsed Logical Plan ==
'Aggregate ['TRANSACTION_TAG], ['TRANSACTION_TAG, count(1) AS count#253L]
+- Project [COMMISSION_GROSSISTE_FIXE#17, COMMISSION_AGENT_REEL#16, TRANSACTION_TAG#33, TETE_DE_PONT#32]
   +- Relation [COMMISSION_AGENT_REEL#16,COMMISSION_GROSSISTE_FIXE#17,COMMISSION_GROSSISTE_REEL#18,COMPTE_PDV#19,COMPTE_REVENDEUR#20,LAST_DAY_OF_WEEK#21,OBJECTIF_AGENT#22,OBJECTIF_GROSSISTE#23,OBJECTIF_REALISE_AGENT#24,OBJECTIF_REALISE_GROSSISTE#25,OBJECTIF_SEMAINE#26,PRIME_HEBDOMADAIRE_GROSSISTE#27,R/O_AGENT#28,R/O_GROSSISTE#29,RUNTIME#30,SEMAINE#31,TETE_DE_PONT#32,TRANSACTION_TAG#33] csv

== Analyzed Logical Plan ==
TRANSACTION_TAG: string, count: bigint
Aggregate [TRANSACTION_TAG#33], [TRANSACTION_TAG#33, count(1) AS count#253L]
+- Project [COMMISSION_GROSSISTE_FIXE#17, COMMISSION_AGENT_REEL#16, TRANSACTION_TAG#33, TETE_DE_PONT#32]
   +- Relation [COMMISSION_AGENT_REEL#16,COMMISSION_GROSSISTE_FIXE#17,COMMISSION_GROSSISTE_REEL#18,COMPTE_PDV#19,COMPTE_REVENDEUR#20,LAST_DAY_OF_WEEK#21

In [13]:
writing_path = "/Users/diakite/Downloads/start/first-bucket/random_objectif_hebdomadaire_grossiste_202211_test"

In [17]:
df_spark \
.write \
.mode("overwrite") \
.format("csv") \
.option("delimiter", "|") \
.option("header", "true")  \
.save(writing_path)